In [1]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

from taco_dataset import CoCoDatasetForYOLO
from config import Config

cfg = Config()

train_transforms = A.Compose(
    [
        A.LongestMaxSize(max_size=int(cfg.IMAGE_SIZE * cfg.scale)),
        A.PadIfNeeded(
            min_height=int(cfg.IMAGE_SIZE * cfg.scale),
            min_width=int(cfg.IMAGE_SIZE * cfg.scale),
            border_mode=cv2.BORDER_CONSTANT,
        ),
        A.RandomCrop(width=cfg.IMAGE_SIZE, height=cfg.IMAGE_SIZE),
        A.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0.6, p=0.4),
        A.OneOf(
            [
                A.ShiftScaleRotate(
                    rotate_limit=20, p=0.5, border_mode=cv2.BORDER_CONSTANT
                ),
                A.Affine(shear=15, p=0.5, mode=cv2.BORDER_CONSTANT),
            ],
            p=1.0,
        ),
        A.HorizontalFlip(p=0.5),
        A.Blur(p=0.1),
        A.CLAHE(p=0.1),
        A.Posterize(p=0.1),
        A.ToGray(p=0.1),
        A.ChannelShuffle(p=0.05),
        A.Normalize(mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255,),
        ToTensorV2(),
    ],

    # got this error
    # Expected y_min for bbox (0.5355392156862745, -0.00015318627450980338, 0.6523692810457516, 0.1803002450980392, 0) to be in the range [0.0, 1.0], got -0.00015318627450980338.
    # rounding issue :/
    # the insane solution to this problem (modifying the library code) https://github.com/albumentations-team/albumentations/issues/459#issuecomment-734454278
    bbox_params=A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[],),
)

train_dataset = CoCoDatasetForYOLO(
    root=cfg.DATASET_PATH,
    annFile=cfg.anns_file_path,
    transform=train_transforms,
    S=cfg.SPLIT_SIZE, B=cfg.NUM_BOXES, C=cfg.NUM_CLASSES
)

train_percentage = 0.8




loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [7]:
from utils import get_stratified_indices

train_indices, test_indices = get_stratified_indices(cfg.anns_file_path, len(train_dataset), train_percentage)

print(train_indices)
print(len(train_indices))
print()
print(test_indices)
print(len(test_indices))

tensor([1120,  820, 1119,  ...,  280,  615,  634])
1199

tensor([ 869, 1104, 1103, 1118,  819,  839,  822,  108,   43, 1151,  408,   34,
         795,   69,  787,  705,  791,  519,   15,    1,  119,   45, 1123,   84,
          47,  430,  532, 1132, 1462, 1364,    7, 1241,  656,  612,  388,  413,
         843,  164,  383, 1464,  362, 1498,  182, 1124, 1135,  613,  628, 1190,
        1411,  114, 1359, 1313,  632,  212,  564,  381,  166, 1063, 1320,  520,
        1329, 1311,  410,  814,  531, 1170, 1312,  324,  261,  556,  389, 1361,
         713,  265,  256,  169,  875,  263,  730, 1309, 1020,  682,  266, 1420,
         718,  213,  456,  665,  283, 1323,  464,  993,  314,  365,  997,   56,
        1212,  307,  973, 1263, 1262,  918,  907, 1259,  507, 1286, 1294,  984,
         940,  923,  899,  778,  788,  917,  107,  407, 1432,  975,  908,  910,
        1181,  896, 1310, 1196,  915,  972,  930,  906,  946,  989, 1003, 1292,
          98,   99, 1470,  275,  379,  702,  239, 1040,  217, 1

In [16]:
import pandas as pd
import datetime

curr_datetime = datetime.datetime.now()

df = pd.DataFrame(train_indices, columns=['train_indices'])
df.to_csv(f"train_indices-{curr_datetime.strftime('%Y-%d-%m_%H-%M-%S')}.csv", index=False)

df2 = pd.DataFrame(test_indices, columns=['test_indices'])
df2.to_csv(f"test_indices-{curr_datetime.strftime('%Y-%d-%m_%H-%M-%S')}.csv", index=False)


In [17]:
df3 = pd.read_csv(f"train_indices-{curr_datetime.strftime('%Y-%d-%m_%H-%M-%S')}.csv")
train_indices2 = torch.tensor(df3['train_indices'].values)

print(train_indices == train_indices2)

tensor([True, True, True,  ..., True, True, True])
